# 1-2,图片数据建模流程范例

In [ ]:
import os

#mac系统上pytorch和matplotlib在jupyter中同时跑需要更改环境变量
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [ ]:
!pip install torchvison
!pip install torchmetrics

In [ ]:
# 导入操作系统库，用于与操作系统进行交互

# 导入PyTorch深度学习框架，用于构建和训练神经网络模型
import torch

# 导入 torchvision 库，提供了用于计算机视觉任务的数据集、模型和工具
import torchvision

# 导入 torchkeras 库，提供了一些增强PyTorch的工具和扩展
import torchkeras

# 导入 torchmetrics 库，提供了用于评估模型性能的度量指标
import torchmetrics

print("torch.__version__ = ", torch.__version__)
print("torchvision.__version__ = ", torchvision.__version__)
print("torchkeras.__version__ = ", torchkeras.__version__)
print("torchmetrics.__version__ = ", torchmetrics.__version__)

### 一，准备数据

cifar2数据集为cifar10数据集的子集，只包括前两种类别airplane和automobile。

训练集有airplane和automobile图片各5000张，测试集有airplane和automobile图片各1000张。

cifar2任务的目标是训练一个模型来对飞机airplane和机动车automobile两种图片进行分类。

我们准备的Cifar2数据集的文件结构如下所示。

![](./data/cifar2.jpg)

在Pytorch中构建图片数据管道通常有两种方法。

第一种是使用torchvision中的datasets.ImageFolder来读取图片然后用DataLoader来并行加载。

第二种是通过继承 torch.utils.data.Dataset 实现用户自定义读取逻辑然后用 DataLoader来并行加载。

第二种方法是读取用户自定义数据集的通用方法，既可以读取图片数据集，也可以读取文本数据集。

本篇我们介绍第一种方法。


In [ ]:
# 从PyTorch库中导入 nn 模块，它包含了神经网络的构建块，如层、激活函数等

# 从 torch.utils.data 库中导入 DataLoader 类，用于批量加载数据
from torch.utils.data import DataLoader

# 从 torchvision 库中导入 transforms 模块并重命名为 T，它用于数据预处理和增强
from torchvision import transforms as T

# 从 torchvision 库中导入 datasets 模块，提供了一些常用的计算机视觉数据集
from torchvision import datasets


In [ ]:
# 定义了一个名为 transform_img 的数据变换函数，用于将图像数据转换为 PyTorch 的张量（tensor）
# T.ToTensor() 将图像数据从 PIL.Image 或 numpy.ndarray 格式转换为 PyTorch 张量
transform_img = T.Compose([T.ToTensor()])


# 定义了一个名为 transform_label 的函数，用于将标签数据转换为 PyTorch 的张量（tensor）
# 这个函数接收一个参数 x，将其包装在一个张量中，并将其数据类型设置为 float
def transform_label(x):
    return torch.tensor([x]).float()


In [ ]:
# 创建训练数据集 ds_train，使用 ImageFolder 数据集，该数据集假定数据按照类别存储在不同的子目录中
# 参数 "./eat_pytorch_datasets/cifar2/train/" 指定训练数据集的根目录
# transform 参数指定对图像数据的转换，使用了之前定义的 transform_img 函数，将图像转换为张量
# target_transform 参数指定对标签数据的转换，使用了之前定义的 transform_label 函数，将标签转换为张量
ds_train = datasets.ImageFolder("./eat_pytorch_datasets/cifar2/train/",
                                transform=transform_img, target_transform=transform_label)

# 创建验证数据集 ds_val，使用 ImageFolder 数据集，该数据集假定数据按照类别存储在不同的子目录中
# 参数 "./eat_pytorch_datasets/cifar2/test/" 指定验证数据集的根目录
# transform 参数指定对图像数据的转换，使用了之前定义的 transform_img 函数，将图像转换为张量
# target_transform 参数指定对标签数据的转换，使用了之前定义的 transform_label 函数，将标签转换为张量
ds_val = datasets.ImageFolder("./eat_pytorch_datasets/cifar2/test/",
                              transform=transform_img, target_transform=transform_label)

# 打印 ds_train 的类别映射，这将显示类别与索引的对应关系
print(ds_train.class_to_idx)


In [ ]:
# 创建训练数据加载器 dl_train，使用 DataLoader 类
# 参数 ds_train 是训练数据集，它将从中加载数据
# batch_size 参数指定每个批次中包含的样本数量，这里设置为 50，表示每个批次加载 50 张图像
# shuffle 参数设置为 True，表示在每个 epoch 开始时随机打乱数据顺序，有助于模型学习
dl_train = DataLoader(ds_train, batch_size=50, shuffle=True)

# 创建验证数据加载器 dl_val，使用 DataLoader 类
# 参数 ds_val 是验证数据集，它将从中加载数据
# batch_size 参数指定每个批次中包含的样本数量，这里设置为 50，表示每个批次加载 50 张图像
# shuffle 参数设置为 False，表示不打乱验证数据的顺序，以确保验证结果的稳定性
dl_val = DataLoader(ds_val, batch_size=50, shuffle=False)


In [ ]:
# 使用 Jupyter Notebook 的魔术命令，将 matplotlib 图形嵌入到输出中
%matplotlib inline

# 配置图形的输出格式为 SVG
%config InlineBackend.figure_format = 'svg'

# 导入 matplotlib 的 pyplot 模块，用于绘制图形
from matplotlib import pyplot as plt

# 创建一个 8x8 英寸大小的图形窗口
plt.figure(figsize=(8, 8))

# 循环遍历并显示前 9 个训练样本
for i in range(9):
    # 从训练数据集中获取图像和标签
    img, label = ds_train[i]

    # 将图像张量的维度重新排列，从 (C, H, W) 转换为 (H, W, C)，以便 matplotlib 显示
    img = img.permute(1, 2, 0)

    # 创建一个子图，并设置标题为对应的标签
    ax = plt.subplot(3, 3, i + 1)

    '''
    当解释以下代码行时：
    这两行代码执行了以下操作：
    1. `img = img.permute(1, 2, 0)`：
        - `img` 是一个图像的张量，通常具有形状 `(C, H, W)`，其中 `C` 表示通道数（例如，彩色图像通常有3个通道：红色、绿色、蓝色），`H` 表示高度，`W` 表示宽度。
        - `permute(1, 2, 0)` 是一个 PyTorch 张量的操作，它重新排列张量的维度。在这里，它将原始张量的维度从 `(C, H, W)` 重新排列为 `(H, W, C)`，这是因为 Matplotlib 需要图像的维度顺序为 `(H, W, C)` 才能正确显示图像。这个操作将通道维度移动到最后一个位置，以便图像在 Matplotlib 中正确渲染。
    2. `ax = plt.subplot(3, 3, i + 1)`：
        - 这行代码创建了一个子图（subplot），在 Matplotlib 中，子图是一个小的图形区域，可以在主图形窗口中容纳多个子图。
        - 参数 `3, 3, i + 1` 指定了子图的网格布局。具体来说，`3, 3` 表示将主图形分成一个3x3的网格，`i + 1` 表示当前子图在这个网格中的位置，`i` 是循环变量，用于表示当前子图的索引。
        - 这个操作在主图形窗口中创建了一个新的子图，并将其赋给变量 `ax`，以便后续对该子图进行设置，如设置标题、图像等。    
    '''

    ax.imshow(img.numpy())  # 显示图像
    ax.set_title("label = %d" % label.item())  # 设置子图标题
    ax.set_xticks([])  # 隐藏 x 轴刻度
    ax.set_yticks([])  # 隐藏 y 轴刻度

# 显示图形窗口
plt.show()


![](./data/1-2-查看样本.png)

In [ ]:
# 遍历训练数据加载器 dl_train，以获取批次数据
for features, labels in dl_train:
    # 打印当前批次中图像数据的形状和标签数据的形状
    # features 是图像数据的批次，labels 是对应的标签批次
    print("图像数据形状:", features.shape)
    print("标签数据形状:", labels.shape)
    break  # 仅打印第一个批次以避免输出过多

'''
这部分代码用于检查训练数据加载器 `dl_train` 中的批次数据的形状。下面是对这部分代码的注释：
在 PyTorch 中，默认的图像数据排列顺序是 Batch（批次大小）、Channel（通道数，如 RGB 通道为 3）、Width（宽度）、Height（高度）。
`features.shape` 中的第一个维度表示批次大小，第二个维度表示通道数，第三和第四个维度表示图像的宽度和高度，打印的形状为 `[50, 3, 32, 32]` 表示一个批次包含 50 张图像，每张图像具有 3 个通道（RGB），宽度和高度分别为 32 像素。
标签数据的形状中，第一个维度表示批次大小，第二个维度为 1，因为每个标签都是一个标量值，标签数据的形状为 `[50, 1]` 表示一个批次包含 50 个标签。
'''


### 二，定义模型

使用Pytorch通常有三种方式构建模型：使用nn.Sequential按层顺序构建模型，继承nn.Module基类构建自定义模型，继承nn.Module基类构建模型并辅助应用模型容器(nn.Sequential,nn.ModuleList,nn.ModuleDict)进行封装。

此处选择通过继承nn.Module基类构建自定义模型。

In [ ]:
# 导入 PyTorch 中的 nn 模块
from torch import nn

# 创建一个自适应最大池化层，指定输出大小为 (1, 1)
pool = nn.AdaptiveMaxPool2d((1, 1))

# 创建一个随机张量 t，大小为 (10, 8, 32, 32)
t = torch.randn(10, 8, 32, 32)

# 使用自适应最大池化层对张量 t 进行池化操作
# 自适应最大池化会根据指定的输出大小自动计算池化核大小，并对输入进行池化
# 这里的输出大小为 (1, 1)，表示在宽度和高度上都进行了自适应的最大池化
output = pool(t)

# 打印池化后的张量形状
print(output.shape)

In [ ]:
# 导入 PyTorch 的 nn 模块
from torch import nn


# 定义一个名为 Net 的神经网络类，继承自 nn.Module
class Net(nn.Module):

    # 构造函数，在这里定义了神经网络的各个层
    def __init__(self):
        super(Net, self).__init__()

        # 第一个卷积层，输入通道数为 3，输出通道数为 32，卷积核大小为 3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)

        # 最大池化层，池化核大小为 2x2，步幅为 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 第二个卷积层，输入通道数为 32，输出通道数为 64，卷积核大小为 5x5
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)

        # 二维 Dropout 层，用于随机丢弃部分神经元，防止过拟合
        self.dropout = nn.Dropout2d(p=0.1)

        # 自适应最大池化层，输出大小为 (1, 1)
        self.adaptive_pool = nn.AdaptiveMaxPool2d((1, 1))

        # 将输入张量展平为一维张量
        self.flatten = nn.Flatten()

        # 第一个全连接层，输入特征数为 64，输出特征数为 32
        self.linear1 = nn.Linear(64, 32)

        # ReLU 激活函数层
        self.relu = nn.ReLU()

        # 第二个全连接层，输入特征数为 32，输出特征数为 1
        self.linear2 = nn.Linear(32, 1)

    # 前向传播函数，定义了数据在模型中的传递方式
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = self.adaptive_pool(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x


# 创建一个 Net 类的实例，即神经网络模型
net = Net()

# 打印神经网络的结构
print(net)


这个神经网络模型包含多个不同类型的层，每个层都有特定的作用和设置缘由。以下是每个层的解释和设置缘由：

1. **卷积层 `self.conv1`**：
   - 作用：卷积层用于从输入图像中提取特征，通过卷积操作可以检测图像中的边缘、纹理等局部特征。
   - 设置缘由：这里的第一个卷积层具有3个输入通道（对应彩色图像的RGB通道）和32个输出通道，使用3x3的卷积核。这样的设置可以帮助模型学习图像的低级特征。

2. **最大池化层 `self.pool`**：
   - 作用：最大池化层用于减小特征图的大小，保留最显著的特征，同时降低计算复杂度。
   - 设置缘由：池化核大小为2x2，步幅为2，这意味着特征图的大小将减小一半。这有助于减少模型参数，同时提取显著特征。

3. **卷积层 `self.conv2`**：
   - 作用：第二个卷积层进一步提取图像的高级特征，对先前卷积层的输出进行处理。
   - 设置缘由：这里的第二个卷积层有32个输入通道（与前一层的输出通道数相同）和64个输出通道，使用5x5的卷积核。增加输出通道数和卷积核大小有助于捕获更复杂的特征。

4. **二维 Dropout 层 `self.dropout`**：
   - 作用：Dropout 层用于在训练过程中随机丢弃部分神经元，以减少过拟合。
   - 设置缘由：设置丢弃率为0.1，表示在每个训练批次中，有10%的神经元被随机丢弃。这有助于提高模型的泛化能力。

5. **自适应最大池化层 `self.adaptive_pool`**：
   - 作用：自适应最大池化层将特征图的大小调整为固定大小，不受输入图像大小的影响。
   - 设置缘由：输出大小设置为(1, 1)，这将确保无论输入图像的大小如何，都会得到固定大小的特征表示，用于后续全连接层的输入。

6. **Flatten 层 `self.flatten`**：
   - 作用：将多维特征张量展平为一维张量，以便连接到全连接层。
   - 设置缘由：这一步是为了将自适应最大池化层的输出变成一维向量，以便输入到全连接层。

7. **全连接层 `self.linear1` 和 `self.linear2`**：
   - 作用：全连接层用于将卷积层和池化层提取的特征映射到最终的输出标签。
   - 设置缘由：第一个全连接层有64个输入特征，32个输出特征；第二个全连接层有32个输入特征，1个输出特征（用于二分类问题）。这些全连接层将学习将高维特征映射到最终输出的关系。

8. **ReLU 激活函数层 `self.relu`**：
   - 作用：ReLU（Rectified Linear Unit）激活函数用于引入非线性性，帮助模型学习更复杂的特征表示。
   - 设置缘由：在全连接层之间使用ReLU激活函数，有助于增强网络的表达能力。

总体来说，这个神经网络模型是一个卷积神经网络（Convolutional Neural Network，CNN），用于处理图像分类任务。通过卷积层和池化层，它可以从图像中提取特征，然后通过全连接层将这些特征映射到最终的分类结果。Dropout 层和ReLU激活函数用于提高模型的泛化能力和非线性表示能力。自适应最大池化层确保了输入图像大小的不变性。不同层的设置经过调整，以在训练过程中有效地学习特定任务的特征表示。这个模型适用于二分类问题，其中最后一个全连接层输出一个标量用于分类。

In [ ]:
# 导入 torchkeras 库，用于模型摘要的打印
import torchkeras

# 使用 torchkeras 的 summary 函数打印模型摘要
# 参数 net 是要打印摘要的神经网络模型
# 参数 input_data 是输入数据的样本，用于确定输入形状
torchkeras.summary(net, input_data=features)


### 三，训练模型

Pytorch通常需要用户编写自定义训练循环，训练循环的代码风格因人而异。

有3类典型的训练循环代码风格：脚本形式训练循环，函数形式训练循环，类形式训练循环。

此处介绍一种较通用的仿照Keras风格的函数形式的训练循环。

该训练循环的代码也是torchkeras库的核心代码。

torchkeras详情:  https://github.com/lyhue1991/torchkeras 


In [ ]:
# 导入所需的 Python 库和模块
import sys
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm
import torch
from torch import nn
from copy import deepcopy


# 定义一个打印日志的函数，用于输出训练过程中的信息
def printlog(info):
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("\n" + "==========" * 8 + "%s" % nowtime)
    print(str(info) + "\n")


# 定义一个用于单步训练或验证的类
class StepRunner:
    # 构造函数
    def __init__(self, net, loss_fn, stage="train", metrics_dict=None, optimizer=None):
        self.net, self.loss_fn, self.metrics_dict, self.stage = net, loss_fn, metrics_dict, stage
        self.optimizer = optimizer

    # 单步训练或验证的方法
    def step(self, features, labels):
        # 计算损失
        preds = self.net(features)
        loss = self.loss_fn(preds, labels)

        # 反向传播与优化
        if self.optimizer is not None and self.stage == "train":
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

        # 计算度量指标
        step_metrics = {self.stage + "_" + name: metric_fn(preds, labels).item()
                        for name, metric_fn in self.metrics_dict.items()}
        return loss.item(), step_metrics

    # 训练步骤
    def train_step(self, features, labels):
        self.net.train()  # 设置模型为训练模式，以便 dropout 层生效
        return self.step(features, labels)

    # 评估步骤
    @torch.no_grad()
    def eval_step(self, features, labels):
        self.net.eval()  # 设置模型为评估模式，以便 dropout 层不生效
        return self.step(features, labels)

    def __call__(self, features, labels):
        if self.stage == "train":
            return self.train_step(features, labels)
        else:
            return self.eval_step(features, labels)


# 定义一个用于整个 epoch 训练或验证的类
class EpochRunner:
    def __init__(self, steprunner):
        self.steprunner = steprunner
        self.stage = steprunner.stage

    def __call__(self, dataloader):
        total_loss, step = 0, 0
        loop = tqdm(enumerate(dataloader), total=len(dataloader), file=sys.stdout)
        for i, batch in loop:
            loss, step_metrics = self.steprunner(*batch)
            step_log = dict({self.stage + "_loss": loss}, **step_metrics)
            total_loss += loss
            step += 1
            if i != len(dataloader) - 1:
                loop.set_postfix(**step_log)
            else:
                epoch_loss = total_loss / step
                epoch_metrics = {self.stage + "_" + name: metric_fn.compute().item()
                                 for name, metric_fn in self.steprunner.metrics_dict.items()}
                epoch_log = dict({self.stage + "_loss": epoch_loss}, **epoch_metrics)
                loop.set_postfix(**epoch_log)

                for name, metric_fn in self.steprunner.metrics_dict.items():
                    metric_fn.reset()
        return epoch_log


# 定义一个函数来训练神经网络模型
def train_model(net, optimizer, loss_fn, metrics_dict,
                train_data, val_data=None,
                epochs=10, ckpt_path='checkpoint.pt',
                patience=5, monitor="val_loss", mode="min"):
    history = {}

    for epoch in range(1, epochs + 1):
        printlog("Epoch {0} / {1}".format(epoch, epochs))

        # 1，训练 -------------------------------------------------
        train_step_runner = StepRunner(net=net, stage="train",
                                       loss_fn=loss_fn, metrics_dict=deepcopy(metrics_dict),
                                       optimizer=optimizer)
        train_epoch_runner = EpochRunner(train_step_runner)
        train_metrics = train_epoch_runner(train_data)

        for name, metric in train_metrics.items():
            history[name] = history.get(name, []) + [metric]

        # 2，验证 -------------------------------------------------
        if val_data:
            val_step_runner = StepRunner(net=net, stage="val",
                                         loss_fn=loss_fn, metrics_dict=deepcopy(metrics_dict))
            val_epoch_runner = EpochRunner(val_step_runner)
            with torch.no_grad():
                val_metrics = val_epoch_runner(val_data)
            val_metrics["epoch"] = epoch
            for name, metric in val_metrics.items():
                history[name] = history.get(name, []) + [metric]

        # 3，早停 -------------------------------------------------
        arr_scores = history[monitor]
        best_score_idx = np.argmax(arr_scores) if mode == "max" else np.argmin(arr_scores)
        if best_score_idx == len(arr_scores) - 1:
            torch.save(net.state_dict(), ckpt_path)
            print("<<<<<< reach best {0} : {1} >>>>>>".format(monitor,
                                                              arr_scores[best_score_idx]), file=sys.stderr)
        if len(arr_scores) - best_score_idx > patience:
            print("<<<<<< {} without improvement in {} epoch, early stopping >>>>>>".format(
                monitor, patience), file=sys.stderr)
            break
        net.load_state_dict(torch.load(ckpt_path))

    return pd.DataFrame(history)


In [ ]:
import torchmetrics


# 自定义准确度度量指标类，继承自 torchmetrics.Accuracy
class Accuracy(torchmetrics.Accuracy):
    def __init__(self, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)

    def update(self, preds: torch.Tensor, targets: torch.Tensor):
        # 在计算准确度时，使用 sigmoid 函数将预测值转换为概率形式，然后进行计算
        super().update(torch.sigmoid(preds), targets.long())

    def compute(self):
        return super().compute()


# 定义损失函数、优化器和度量指标
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

# 创建度量指标字典，包括自定义的准确度度量指标
metrics_dict = {"acc": Accuracy(task='binary')}

# 使用 train_model 函数进行模型训练
dfhistory = train_model(net,
                        optimizer,
                        loss_fn,
                        metrics_dict,
                        train_data=dl_train,
                        val_data=dl_val,
                        epochs=10,
                        patience=5,
                        monitor="val_acc",
                        mode="max")


### 四，评估模型

In [ ]:
dfhistory

In [ ]:
# 导入绘图库
import matplotlib.pyplot as plt


# 定义绘制度量指标曲线的函数
def plot_metric(dfhistory, metric):
    # 从训练历史数据帧中提取训练和验证的度量指标值
    train_metrics = dfhistory["train_" + metric]
    val_metrics = dfhistory['val_' + metric]

    # 创建 x 轴的 epoch 数组
    epochs = range(1, len(train_metrics) + 1)

    # 绘制训练集和验证集度量指标的曲线
    plt.plot(epochs, train_metrics, 'bo--')  # 训练集度量指标的蓝色圆点线
    plt.plot(epochs, val_metrics, 'ro-')  # 验证集度量指标的红色实线

    # 设置图表标题和轴标签
    plt.title('Training and validation ' + metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)

    # 添加图例，标明曲线含义
    plt.legend(["train_" + metric, 'val_' + metric])

    # 显示图表
    plt.show()


In [ ]:
plot_metric(dfhistory, "loss")

In [ ]:
plot_metric(dfhistory, "acc")

### 五，使用模型

In [ ]:
# 定义一个函数用于进行预测
def predict(net, dl):
    # 设置模型为评估模式，以便 dropout 层不生效
    net.eval()

    # 使用 torch.no_grad() 上下文管理器，关闭梯度计算，加速预测过程
    with torch.no_grad():
        # 对数据加载器中的每个样本进行预测，并将结果拼接成一个张量
        result = nn.Sigmoid()(torch.cat([net.forward(t[0]) for t in dl]))

    # 返回预测结果的数据
    return result.data


In [ ]:
#预测概率
y_pred_probs = predict(net, dl_val)
y_pred_probs

In [ ]:
#预测类别
y_pred = torch.where(y_pred_probs > 0.5,
                     torch.ones_like(y_pred_probs), torch.zeros_like(y_pred_probs))
y_pred

### 六，保存模型

推荐使用保存参数方式保存Pytorch模型。

In [ ]:
print(net.state_dict().keys())  #输出模型状态字典中的键的列表，每个键对应一个模型的参数或层的名称。这些键通常包括卷积层的权重、偏置项、线性层的权重、偏置项等等。查看这些键可以帮助您了解模型的结构和参数情况。

In [ ]:
# 保存模型参数到文件
torch.save(net.state_dict(), "./data/net_parameter.pt")

# 创建一个新的模型对象并加载保存的参数
net_clone = Net()
net_clone.load_state_dict(torch.load("./data/net_parameter.pt"))

# 使用加载后的模型进行预测
y_pred_probs_clone = predict(net_clone, dl_val)

# 输出预测结果
print(y_pred_probs_clone)
